# Assignment 3  
## Applied Machine Learning

Andrew Chan 
EBE869

This assignment uses the `Suicide Rates Overview 1985 to 2016` dataset from Kaggle: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016. This assumes that you have downloaded the dataset in `master.csv` which is in the same directory as this notebook.

In [1]:
import pandas as pd
import math
import numpy as np

# Locate and load the data file
df = pd.read_csv('master.csv')

# Sanity check
print(f'N rows={len(df)}, M columns={len(df.columns)}')
df.head()

N rows=27820, M columns=12


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [2]:
df.columns.values

array(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype=object)

Adjust the gdp_for_year ($) from string to float.

In [3]:
df[' gdp_for_year ($) ']

0         2,156,624,900
1         2,156,624,900
2         2,156,624,900
3         2,156,624,900
4         2,156,624,900
              ...      
27815    63,067,077,179
27816    63,067,077,179
27817    63,067,077,179
27818    63,067,077,179
27819    63,067,077,179
Name:  gdp_for_year ($) , Length: 27820, dtype: object

In [4]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].str.replace(',', '')

In [5]:
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].astype(float) 

In [6]:
df[' gdp_for_year ($) '] 

0        2.156625e+09
1        2.156625e+09
2        2.156625e+09
3        2.156625e+09
4        2.156625e+09
             ...     
27815    6.306708e+10
27816    6.306708e+10
27817    6.306708e+10
27818    6.306708e+10
27819    6.306708e+10
Name:  gdp_for_year ($) , Length: 27820, dtype: float64

# Preprocessing (from Module 3)

The assingment states to use the "let's get back to the pre-processed dataset Suicide Rates Overview 1985 to 2016 file", thus we repeat the preprocessing steps here:

## Preprocessing Steps:

### Convert dependent variable to binary values

In [7]:
suicides_mean = df["suicides/100k pop"].mean()
suicides_mean

12.816097411933894

We will create a modify `suicides/100k pop` as follows:  
* All values larger than or equal to `suicides_mean` will be labelled with `high suicide rate` 
* All values smaller than `suicides_mean` will be labelled with `low suicide rate` 

In [8]:
def isHigh(x, mean_val):
    if x > mean_val:
        return 'high suicide rate'
    else:
        return 'low suicide rate'
df['suicides/100k pop'] = df['suicides/100k pop'].apply(isHigh, args=[suicides_mean])
df['suicides/100k pop'].value_counts()

low suicide rate     19061
high suicide rate     8759
Name: suicides/100k pop, dtype: int64

In [9]:
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,low suicide rate,Albania1987,NaN,2.156625e+09,796,Generation X
1,Albania,1987,male,35-54 years,16,308000,low suicide rate,Albania1987,NaN,2.156625e+09,796,Silent
2,Albania,1987,female,15-24 years,14,289700,low suicide rate,Albania1987,NaN,2.156625e+09,796,Generation X
3,Albania,1987,male,75+ years,1,21800,low suicide rate,Albania1987,NaN,2.156625e+09,796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,low suicide rate,Albania1987,NaN,2.156625e+09,796,Boomers


### Removal redundant features

#### Remove `suicides_no`

`suicides_no` is highly correlated with with `suicides/100k pop`. Since `suicides/100k pop` was chosen as the dependent variable, it does not make sense to use `suicides_no` since `suicides/100k pop` is $$\frac{suicides\_no}{population/100000}$$ which is a derived attribute. 

Another reason why I chose to keep `suicides/100k pop` is because it helps **normalize suicides by population**. A large country may have many suicides because there are more people that live there, but a smaller country may have more suicides per population. 

In [10]:
df = df.drop(['suicides_no'], axis=1)

#### Remove `gdp_for_year ($)`

`gdp_for_year ($)` is highly correlated with with `gdp_per_capita ($)`. Since `gdp_per_capita ($)` was chosen as the dependent variable, it does not make sense to use `gdp_for_year ($)` since `gdp_per_capita ($)` is $$\frac{gdp\_for\_year ($)}{population}$$ which is a derived attribute.


Another reason why I chose to keep `gdp_per_capita ($)` is because it helps **normalize GDP by population**. A large country may have more gross domestic product because there are more people that live there, but a smaller country may have more gdp per capita. 

In [11]:
df = df.drop([' gdp_for_year ($) '], axis=1)

#### Remove `country-year`

Since `country-year` already is derived from `country` and `year`, it is redundant to include the `country-year`. Thus, we drop `country-year`.

I also chose to keep `country` and `year` because upon one-hot encoding, there will be **N countries** + **M years** columns versus **N x M** additional columns, which should help with computational speed. 

In [12]:
df = df.drop(['country-year'], axis=1)

### Mean imputation

For all values with NaN, we will replace with mean so that we do not lose information.

In [13]:
df = df.fillna(df.mean())

In [14]:
final_features = df.columns

### One hot encoding of all nominal features

In [15]:
df_no_target = pd.get_dummies(df[['country','year','sex','age','population','HDI for year','gdp_per_capita ($)','generation']], drop_first = True)

### Normalization and Standardization

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_no_target[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']] = scaler.fit_transform(df[['year', 'population', 'HDI for year', 'gdp_per_capita ($)']])

## Final list of features selected

* 'country'
* 'year'
* 'sex'
* 'age'
* 'population'
* 'suicides/100k pop'
* 'HDI for year' 
* 'gdp_per_capita ($)'
* 'generation'

# 1. [20 pts] Due to the severity of this real-world crisis, what information would be the most important one to "machine learn"? Can it be learned?

The most important target variable to machine learn is `suicides/100k pop` as we would like to be able to predict low versus high suicide rates. We would also like to determine what factors may lead to higher suicide rates. Predicting this variable will allow countries to issue multilateral policies to help reduce future deaths. For example, it if it found that `age` is a large factor in suicide rates, countries can focus on preventative social wellness programs for those specific groups. 

# 1. [20 pts] What is the dependent variable you decided? Why?

The dependent variable will be generated from the `suicides/100k pop`. We have chosen to use this versus `suicides_no` because this is normalized to population versus an absolute number of suicides. This will help provide a better prediction so that the model does not skew towards larger countries which may naturally have more suicides. 

# 2. [20 pts] Set the dependent variable into two categories based on a defensible criteria. 
(Hint: skirts of the probability density function)

To attempt a machine learning solution we will turn this into a binary classification problem of `low suicide rate` versus `high suicide rate`. Framing the problem this way will allow us to train a classifier. 

The steps to take will be the following:
1. We will use the dependent variable `suicides/100k pop`
2. Then calculate the mean of this feature `suicides_mean`
3. Convert the data into a binary classification problem such that   
    a. If the value is > than `suicides_mean` then convert to **`high suicide rate`**  
    b. If the value is < than `suicides_mean` then convert to **`low suicide rate`**   

# 3. [20 pts] Develop your classification model(s) to solve your defined problem.

We will create a model using svc and a train test split of 0.80 and 0.20.

Set the x and y values:

In [17]:
df_y = df['suicides/100k pop']
df_X = df_no_target
y = df_y.values
X = df_X.values

In [18]:
from sklearn.model_selection import train_test_split

Use the default train test split of 0.75 and 0.25.

In [19]:
X_train, X_test, y_train, expected = train_test_split(X, y, random_state=0)

Use linear SVC which is a linear svm that uses one versus rest scheme:

In [20]:
from sklearn.svm import LinearSVC
from sklearn import metrics
clf = LinearSVC(random_state=0,max_iter=4000)
clf.fit(X_train, y_train)

LinearSVC(max_iter=4000, random_state=0)

In [21]:
predicted = clf.predict(X_test)

# 4. [20 pts] Evaluate (and report) the model performance(s) using some of the standard techniques (e.g. 80-20 split, 10-fold cross validation, etc.).

In [22]:
print(f'Classification report for classifier {clf}:\n{metrics.accuracy_score(expected, predicted)}\n')

Classification report for classifier LinearSVC(max_iter=4000, random_state=0):
0.906254493170381



# 5. [20 pts] Using your classifier model, what is the predicted category of your dependent variable for the input: "year=2000, generation=Generation X, age=20, gender=male"?

# 5. [20 pts] Using your classifier model, what is the predicted category of your dependent variable for the input: "year=2000, generation=Generation X, age=20, gender=male"?